<a href='https://aims-senegal.org/'> <img src='http://sn.nexteinstein.org/wp-content/uploads/sites/12/2016/07/aims_senegal.jpg' /></a>

# **Feature Selection: Filter Method**

In this notebook we look into the problem of feature selection, in which we have a large pool of ***d*** features and our goal is to select a small number of features ***k*** that will be used by our model. So our goal is to learn a model that only relies on ***k << d*** features.



## **Why do we do feature selection ?**
Models that use only a small subset of features require a
smaller memory footprint and can be applied faster. Furthermore, in applications such as medical diagnostics, obtaining each possible “feature” (e.g., test result) can be costly; therefore, a model that uses only a small number of features is desirable even at the cost of a small degradation in performance, relative to a model that uses more features.


## **How do we do feature selection ?**

Ideally, we could have tried all subsets of ***k*** out of ***d*** features and choose the subset which leads to the best performing model (using cross validation). However, such an exhaustive search is usually computationally intractable.


## **Feature selection methods:**

Moustapha covered 3 computationally feasible approaches for feature selection: L1 Regularization, Wrapper (Greedy) Methods (Forward Search & Backward Elimination) and Filter Method. While these methods
cannot guarantee finding the optimal subset, they often work reasonably well in practice.

## **Filter Method**
One of the simplest approach for feature selection is the filter method, in which we assess individual features, independently of other features, according to some quality measure. We can then select the ***k*** features that achieve the highest score (alternatively, we can decide on the number of features to select according to the value of their scores).

Many quality measures for features have been proposed in the literature. We will use [Mutual Information](https://en.wikipedia.org/wiki/Mutual_information) between the feature and the target as our measure of feature quality. Mutual information between a pair of random variables ***X*** and ***Y*** is defined as:

<img src='https://wikimedia.org/api/rest_v1/media/math/render/svg/4b37af4015d70867b9c50b9cf82e0fc3598b070c' />

Mutual information (MI) between two random variables is a non-negative value, which measures the dependency between the variables. It is equal to zero if and only if two random variables are independent, and higher values mean higher dependency.

Where D<sub>KL</sub> is the [Kullback Leibler Divergence](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence) which is a measure of how one probability distribution is different from another. 


---
---
---
---
---

## **Setup Code**

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import linear_model
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.model_selection import cross_val_score, train_test_split, KFold 
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import mutual_info_regression

---
---
---
---
---

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Data** 

We will experiment **again** with cancer mortality rates dataset from [here](https://data.world/nrippner/ols-regression-challenge). A slightly modified copy was shared with you on workplace a few days ago. Please Load the data "cancer-mortality-rate.csv" to your runtime or mount your drive and load it. The target is the column named **"TARGET_deathRate"**.

In [4]:
data = pd.read_csv('/content/drive/MyDrive/tut/cancer-mortality-rate.csv', index_col=0)
data.head()

,avgAnnCount,avgDeathsPerYear,TARGET_deathRate,incidenceRate,medIncome,popEst2015,povertyPercent,studyPerCap,MedianAge,MedianAgeMale,MedianAgeFemale,AvgHouseholdSize,PercentMarried,PctNoHS18_24,PctHS18_24,PctSomeCol18_24,PctBachDeg18_24,PctHS25_Over,PctBachDeg25_Over,PctEmployed16_Over,PctUnemployed16_Over,PctPrivateCoverage,PctPrivateCoverageAlone,PctEmpPrivCoverage,PctPublicCoverage,PctPublicCoverageAlone,PctWhite,PctBlack,PctAsian,PctOtherRace,PctMarriedHouseholds,BirthRate,avgAnnCount-2,avgAnnCount-3,avgAnnCount-4,avgAnnCount-5,avgAnnCount-6,avgAnnCount-7,avgAnnCount-8,avgAnnCount-9,...,PctBlack-2,PctBlack-3,PctBlack-4,PctBlack-5,PctBlack-6,PctBlack-7,PctBlack-8,PctBlack-9,PctAsian-2,PctAsian-3,PctAsian-4,PctAsian-5,PctAsian-6,PctAsian-7,PctAsian-8,PctAsian-9,PctOtherRace-2,PctOtherRace-3,PctOtherRace-4,PctOtherRace-5,PctOtherRace-6,PctOtherRace-7,PctOtherRace-8,PctOtherRace-9,PctMarriedHouseholds-2,PctMarriedHouseholds-3,PctMarriedHouseholds-4,PctMarriedHouseholds-5,PctMarriedHouseholds-6,PctMarriedHouseholds-7,PctMarriedHouseholds-8,PctMarriedHouseholds-9,BirthRate-2,BirthRate-3,BirthRate-4,BirthRate-5,BirthRate-6,BirthRate-7,BirthRate-8,BirthRate-9
0,1397.0,469,164.9,489.8,61898,260131,11.2,499.748204,39.3,36.9,41.7,2.54,52.5,11.5,39.5,42.1,6.9,23.2,19.6,51.9,8.0,75.1,NaN,41.6,32.9,14.0,81.780529,2.594728,4.821857,1.843479,52.856076,6.118831,1951609.0,2.726398e+09,3.808778e+12,5.320862e+15,7.433245e+18,1.038424e+22,1.450679e+25,2.026598e+28,...,6.732615,17.469307,45.328106,117.614122,305.176694,791.850616,2054.637228,5331.225430,23.250306,112.109653,540.576725,2606.583721,12568.574225,60603.868891,292223.195619,1.409058e+06,3.398413,6.264902,11.549212,21.290724,39.248992,72.354674,133.384289,245.891073,2793.764757,147667.442010,7.805122e+06,4.125481e+08,2.180567e+10,1.152562e+12,6.091992e+13,3.219988e+15,37.440093,229.089604,1401.760576,8577.136107,52482.046553,321128.774915,1.964933e+06,1.202309e+07
1,173.0,70,161.3,411.6,48127,43269,18.6,23.111234,33.0,32.2,33.7,2.34,44.5,6.1,22.4,64.0,7.5,26.0,22.7,55.9,7.8,70.2,53.8,43.6,31.1,15.3,89.228509,0.969102,2.246233,3.741352,45.372500,4.333096,29929.0,5.177717e+06,8.957450e+08,1.549639e+11,2.680875e+13,4.637914e+15,8.023592e+17,1.388081e+20,...,0.939160,0.910142,0.882021,0.854768,0.828358,0.802764,0.777961,0.753923,5.045561,11.333503,25.457684,57.183879,128.448293,288.524742,648.093677,1.455769e+03,13.997711,52.370359,195.935921,733.065158,2742.654452,10261.234432,38390.885152,143633.796942,2058.663796,93406.723998,4.238097e+06,1.922930e+08,8.724816e+09,3.958667e+11,1.796146e+13,8.149565e+14,18.775717,81.356978,352.527560,1527.535610,6618.957797,28680.576760,1.242757e+05,5.384984e+05
2,102.0,50,174.7,349.7,49348,21026,14.6,47.560164,45.0,44.0,45.8,2.62,54.2,24.0,36.6,NaN,9.5,29.0,16.0,45.9,7.0,63.7,43.5,34.9,42.1,21.1,90.922190,0.739673,0.465898,2.747358,54.444868,3.729488,10404.0,1.061208e+06,1.082432e+08,1.104081e+10,1.126162e+12,1.148686e+14,1.171659e+16,1.195093e+18,...,0.547117,0.404688,0.299337,0.221411,0.163772,0.121138,0.089602,0.066277,0.217061,0.101128,0.047116,0.021951,0.010227,0.004765,0.002220,1.034235e-03,7.547978,20.736999,56.971967,156.522407,430.023135,1181.427634,3245.805027,8917.389410,2964.243692,161387.857618,8.786741e+06,4.783929e+08,2.604604e+10,1.418073e+12,7.720681e+13,4.203515e+15,13.909079,51.873742,193.462489,721.515996,2690.885118,10035.623263,3.742773e+04,1.395863e+05
3,427.0,202,194.8,430.4,44243,75882,17.1,342.637253,42.8,42.2,43.4,2.52,52.7,20.2,41.2,36.1,2.5,31.6,9.3,48.3,12.1,58.4,40.3,35.0,45.3,25.0,91.744686,0.782626,1.161359,1.362643,51.021514,4.603841,182329.0,7.785448e+07,3.324386e+10,1.419513e+13,6.061321e+15,2.588184e+18,1.105155e+21,4.719010e+23,...,0.612503,0.479361,0.375160,0.293610,0.229787,0.179837,0.140745,0.110151,1.348754,1.566387,1.819137,2.112671,2.453569,2.849473,3.309260,3.843238e+00,1.856796,2.530151,3.447693,4.697975,6.401664,8.723184,11.886587,16.197177,2603.194940,132818.948317,6.776624e+06,3.457536e+08,1.764087e+10,

In [5]:
print('Number of Data Points: ', data.shape[0])
print('Number of Features: ', data.shape[1])

Number of Data Points:  3047
Number of Features:  280


---
---
---
---
---

## **Data Preprocessing**

In [6]:
# Deal with NaN values
# We can replace NaN values with the mean value of each feature (We will use this)
# We can also drop the columns with high percentage of NaN values
# We can drop rows that contain NaN Values
# We can use iterpolation techniques to fill the NaNs
# Which one I should use ? I don't know, you have to experiemnt 

for col in data.columns:
    if data[col].isna().sum() != 0:
        print(f"Col: {col}, Number of nan value: {data[col].isna().sum()}")
        data[col][data[col].isna()] = data[col].mean()

Col: PctSomeCol18_24, Number of nan value: 2285
Col: PctEmployed16_Over, Number of nan value: 152
Col: PctPrivateCoverageAlone, Number of nan value: 609
Col: PctSomeCol18_24-2, Number of nan value: 2285
Col: PctSomeCol18_24-3, Number of nan value: 2285
Col: PctSomeCol18_24-4, Number of nan value: 2285
Col: PctSomeCol18_24-5, Number of nan value: 2285
Col: PctSomeCol18_24-6, Number of nan value: 2285
Col: PctSomeCol18_24-7, Number of nan value: 2285
Col: PctSomeCol18_24-8, Number of nan value: 2285
Col: PctSomeCol18_24-9, Number of nan value: 2285
Col: PctEmployed16_Over-2, Number of nan value: 152
Col: PctEmployed16_Over-3, Number of nan value: 152
Col: PctEmployed16_Over-4, Number of nan value: 152
Col: PctEmployed16_Over-5, Number of nan value: 152
Col: PctEmployed16_Over-6, Number of nan value: 152
Col: PctEmployed16_Over-7, Number of nan value: 152
Col: PctEmployed16_Over-8, Number of nan value: 152
Col: PctEmployed16_Over-9, Number of nan value: 152
Col: PctPrivateCoverageAlone-2,

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [7]:
# Train/Test Split with K fold cross validation 
y = data['TARGET_deathRate']
X = data.drop(columns=['TARGET_deathRate'])

In [8]:
kf = KFold(n_splits=5, random_state= 42, shuffle=True) 

# KFold training helper function
def KFold_train(model, X, Y, kf):
    
    scores = []

    for train_index, val_index in kf.split(X): #we split the train to K-Folds
        
        x_train, x_val = X.loc[train_index], X.loc[val_index]
        y_train, y_val = Y.loc[train_index], Y.loc[val_index]

        # Normalize the features
        mu = x_train.mean()
        std = x_train.std()

        x_train_norm = (x_train - mu) / std
        x_val_norm = (x_val - mu) / std

        # fit
        model.fit(x_train_norm, y_train)

        # evaluate
        pred_y = model.predict(x_val_norm)

        # compute the loss
        scores.append(mean_squared_error(y_val, pred_y))
    
    return np.mean(np.array(scores))

---
---
---
---
---

##  **Fit a Linear Regression model on all the features**

In [9]:
# Define the model ===> un regularized linear regression model
model = linear_model.LinearRegression(fit_intercept=True)

# Finding the model params ===> Basically solving OLS problem
# using K-fold cross validation to estimate the perfromance (MSE)
mean_mse = KFold_train(model, X, y, kf)

print(' Mean MSE error of the model: ', mean_mse)

 Mean MSE error of the model:  9508998098727.594


In [10]:
model.coef_

array([-1.37418623e+02,  1.63917545e+02,  4.49598942e+04, -5.66797398e+01,
       -5.57802098e+01, -1.57787210e+03, -1.94659431e+01,  5.01488778e+02,
       -1.82473216e+05,  2.12667682e+05, -1.31499348e+03,  1.74351844e+05,
       -1.62782719e+02, -7.17968432e+02,  1.10258485e+03, -1.35555814e+01,
       -1.98984108e+04,  1.11192148e+02,  2.12399219e+05, -5.94685251e+00,
       -9.17143146e+03, -3.11822920e+04,  5.61521159e+03, -2.08419380e+04,
        5.66216175e+02,  1.30396852e+04,  4.40612691e+01, -6.80737320e+00,
       -3.98538657e+00, -6.14583588e+04, -1.57758564e+02,  2.57773357e+03,
       -2.69990892e+04,  1.78375464e+05, -9.25877257e+05,  3.59255163e+06,
       -8.61174094e+06,  1.05270363e+07, -4.70649792e+06, -2.02734533e+03,
        1.42387201e+04, -4.25412857e+04,  7.26570240e+02, -1.82797488e-01,
        1.47580062e-01,  4.83421110e-01, -2.56955289e-01, -3.64864327e+05,
        1.53682743e+06, -4.54570252e+06,  9.54962927e+06, -1.34557327e+07,
        1.19573433e+07, -

---
---
---
---
---

##  **Fit a Linear Regression model with L1 regularization on all the features**

In [11]:
# Define the model ===> Lasso is just linear regression + l1 reg (check the sklearn docs)
model = linear_model.Lasso(alpha=0.1, fit_intercept=True, max_iter=5000)

# Finding the model params ===> Basically solving the objective (1 / (2 * n_samples)) * ||y - Xw||^2_2 + alpha * ||w||_1
# using K-fold cross validation to estimate the perfromance (MSE)
mean_mse = KFold_train(model, X, y, kf)

print(' Mean MSE error of the model: ', mean_mse)

 Mean MSE error of the model:  497.6300219562386


---
---
---
---
---

let's see how many features are actually used by the L1 regularized model

In [12]:
# to do print the model parameters (Hint: use model.coef_) and see how many of them are larger than 0.1 (in absolute value)
model.coef_
x = model.coef_
sum = x[np.where(abs(x)>0.1)]

print(len(sum))

# Your code 

68


---
---
---
---
---

### **Feature selection with Filter method**



*   Calculate the mutual information score between each feature column and the target column (use: mutual_info_regression from sklearn)
Bonus: if you implemented it yourself.

*   Now it's up to you to decide how to proceed:

*    you can take the top k features based on MI.   

*    you can choose the features that above a specified threshold based on MI scores.

*    you can do further feature selection by doing wrapper methods on top of the k selected features.

**TASK: Train a linear model on your selected set of features and compute the performance, compare your results with the L1 regularized model above.**

P.S: I got a Mean MSE error of a model = 599.05 with training on only 2 features.



In [13]:
mutual_info_regression(X,y)

array([1.26980036e-01, 1.15462043e-01, 1.52929282e-01, 1.56983814e-01,
       9.67380625e-02, 1.69097742e-01, 2.97777185e-02, 1.98784375e-02,
       2.11808640e-02, 4.29647657e-02, 3.66224639e-02, 5.29940130e-02,
       1.12867420e-02, 1.69781254e-02, 1.15855108e-02, 7.94737531e-02,
       1.58555454e-01, 1.91269757e-01, 1.16848148e-01, 9.70365241e-02,
       1.45515764e-01, 8.70782918e-02, 7.08920904e-02, 1.17008213e-01,
       1.69829741e-01, 2.83631812e-02, 7.26680137e-02, 7.04127103e-02,
       2.65009673e-02, 5.78420725e-02, 1.77816868e-02, 7.70104816e-02,
       3.08782151e-02, 2.07083256e-02, 8.18454990e-03, 8.11102609e-03,
       0.00000000e+00, 2.31186977e-03, 3.52863284e-04, 7.09598881e-02,
       3.10619979e-02, 2.51875059e-02, 1.69632731e-02, 3.13409421e-02,
       5.46415809e-02, 3.99242430e-02, 7.92533613e-02, 1.55989562e-01,
       1.61896126e-01, 1.66962606e-01, 1.45142499e-01, 1.24208349e-01,
       1.12208955e-01, 1.02317994e-01, 8.95875331e-02, 1.50905883e-01,
      

In [14]:
X_frame = mutual_info_regression(X,y) > 0.15

In [15]:
X_frame

array([False, False,  True,  True, False,  True, False, False, False,
       False, False, False, False, False, False, False,  True,  True,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,  True,  True,  True, False, False, False, False,
       False,  True,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
        True,  True, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [16]:
new_features = X.columns[X_frame]

In [17]:
new_features

Index(['incidenceRate', 'medIncome', 'povertyPercent', 'PctHS25_Over',
       'PctBachDeg25_Over', 'PctPublicCoverageAlone', 'incidenceRate-2',
       'incidenceRate-3', 'incidenceRate-4', 'medIncome-2', 'medIncome-3',
       'povertyPercent-2', 'povertyPercent-3', 'povertyPercent-4',
       'povertyPercent-6', 'PctHS25_Over-2', 'PctHS25_Over-3',
       'PctBachDeg25_Over-2', 'PctBachDeg25_Over-3', 'PctBachDeg25_Over-4',
       'PctBachDeg25_Over-5', 'PctBachDeg25_Over-6',
       'PctPublicCoverageAlone-2', 'PctPublicCoverageAlone-3',
       'PctPublicCoverageAlone-4', 'PctPublicCoverageAlone-5'],
      dtype='object')

In [18]:
list(new_features)

['incidenceRate',
 'medIncome',
 'povertyPercent',
 'PctHS25_Over',
 'PctBachDeg25_Over',
 'PctPublicCoverageAlone',
 'incidenceRate-2',
 'incidenceRate-3',
 'incidenceRate-4',
 'medIncome-2',
 'medIncome-3',
 'povertyPercent-2',
 'povertyPercent-3',
 'povertyPercent-4',
 'povertyPercent-6',
 'PctHS25_Over-2',
 'PctHS25_Over-3',
 'PctBachDeg25_Over-2',
 'PctBachDeg25_Over-3',
 'PctBachDeg25_Over-4',
 'PctBachDeg25_Over-5',
 'PctBachDeg25_Over-6',
 'PctPublicCoverageAlone-2',
 'PctPublicCoverageAlone-3',
 'PctPublicCoverageAlone-4',
 'PctPublicCoverageAlone-5']

In [19]:
new_features_df = pd.DataFrame(X, columns=list(new_features))

In [20]:
model = linear_model.LinearRegression(fit_intercept=True)

In [21]:
mean_mse = KFold_train(model, new_features_df, y, kf)

print(' Mean MSE error of the model: ', mean_mse)

 Mean MSE error of the model:  395.0555396704641


In [22]:
X_frame = mutual_info_regression(X,y)

In [44]:
K = 22

In [45]:
ind = np.argpartition(X_frame, -K)[-K:]

In [46]:
ind

array([224, 223, 169, 226, 168,  17,  24,  71,  72,  73,  16, 167, 159,
       225, 171,  47,  48,   5,  49,   3,   2, 170])

In [47]:
best_ten_col = [X.columns[i] for i in list(ind)]

In [48]:
best_ten_col

['PctPublicCoverageAlone-3',
 'PctPublicCoverageAlone-2',
 'PctBachDeg25_Over-4',
 'PctPublicCoverageAlone-5',
 'PctBachDeg25_Over-3',
 'PctBachDeg25_Over',
 'PctPublicCoverageAlone',
 'povertyPercent-2',
 'povertyPercent-3',
 'povertyPercent-4',
 'PctHS25_Over',
 'PctBachDeg25_Over-2',
 'PctHS25_Over-2',
 'PctPublicCoverageAlone-4',
 'PctBachDeg25_Over-6',
 'incidenceRate-2',
 'incidenceRate-3',
 'povertyPercent',
 'incidenceRate-4',
 'medIncome',
 'incidenceRate',
 'PctBachDeg25_Over-5']

In [49]:
with_best_ten_features = pd.DataFrame(X, columns=list(best_ten_col))

In [50]:
with_best_ten_features

,PctPublicCoverageAlone-3,PctPublicCoverageAlone-2,PctBachDeg25_Over-4,PctPublicCoverageAlone-5,PctBachDeg25_Over-3,PctBachDeg25_Over,PctPublicCoverageAlone,povertyPercent-2,povertyPercent-3,povertyPercent-4,PctHS25_Over,PctBachDeg25_Over-2,PctHS25_Over-2,PctPublicCoverageAlone-4,PctBachDeg25_Over-6,incidenceRate-2,incidenceRate-3,povertyPercent,incidenceRate-4,medIncome,incidenceRate,PctBachDeg25_Over-5
0,2744.000,196.00,147578.9056,5.378240e+05,7529.536,19.6,14.0,125.44,1404.928,15735.1936,23.2,384.16,538.24,38416.0000,5.669391e+07,239904.040000,1.175050e+08,11.2,5.755395e+10,61898,489.800000,2.892547e+06
1,3581.577,234.09,265523.7841,8.384114e+05,11697.083,22.7,15.3,345.96,6434.856,119688.3216,26.0,515.29,676.00,54798.1281,1.368218e+08,169414.560000,6.973103e+07,18.6,2.870129e+10,48127,411.600000,6.027390e+06
2,9393.931,445.21,65536.0000,4.182272e+06,4096.000,16.0,21.1,213.16,3112.136,45437.1856,29.0,256.00,841.00,198211.9441,1.677722e+07,122290.090000,4.276484e+07,14.6,1.495487e+10,49348,349.700000,1.048576e+06
3,15625.000,625.00,7480.5201,9.765625e+06,804.357,9.3,25.0,292.41,5000.211,85503.6081,31.6,86.49,998.56,390625.0000,6.469902e+05,185244.160000,7.972909e+07,17.1,3.431540e+10,44243,430.400000,6.956884e+04
4,11697.083,515.29,50625.0000,6.027390e+06,3375.000,15.0,22.7,156.25,1953.125,24414.0625,33.4,225.00,1115.56,265523.7841,1.139062e+07,122570.010000,4.291176e+07,12.5,1.502341e+10,49955,350.100000,7.593750e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3042,2299.968,174.24,53379.4816,4.007464e+05,3511.808,15.2,13.2,153.76,1906.624,23642.1376,32.2,231.04,1036.84,30359.5776,1.233280e+07,205707.078287,9.329833e+07,12.4,4.231540e+10,46961,453.549422,8.113681e+05
3043,5545.233,313.29,23642.1376,1.737266e+06,1906.624,12.4,17.7,353.44,6644.672,124919.8336,23.1,153.76,533.61,98150.6241,3.635215e+06,205707.078287,9.329833e+07,18.8,4.231540e+10,48609,453.549422,2.931625e+05
3044,4741.632,282.24,26843.5456,1.338278e+06,2097.152,12.8,16.8,225.00,3375.000,50625.0000,23.0,163.84,529.00,79659.4176,4.398047e+06,205707.078287,9.329833e+07,15.0,4.231540e+10,51144,453.549422,3.435974e+05
3045,2744.000,196.00,42998.1696,5.378240e+05,2985.984,14.4,14.0,176.89,2352.637,31290.0721,36.1,207.36,1303.21,38416.0000,8.916100e+06,205707.078287,9.329833e+07,13.3,4.231540e+10,50745,453.549422,6.191736e+05


In [51]:
model = linear_model.LinearRegression(fit_intercept=True)

In [52]:
mean_mse = KFold_train(model, with_best_ten_features, y, kf)

print(' Mean MSE error of the model: ', mean_mse)

 Mean MSE error of the model:  396.6008394260374
